In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import chess
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import chess.polyglot
import tensorflow as tf

In [3]:
def plot(x,y,x_label,y_label,title):
    fig, ax = plt.subplots()
    ax.plot(x,y, label=y_label)
    ax.legend()
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    plt.show()
def multiPlot(x,ys,x_label,y_label,title):
    fig, ax = plt.subplots()
    for k in range(len(ys)):
        ax.plot(x,ys[k][0], label= ys[k][1])
    ax.legend()
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    plt.show()

In [32]:
class ChessValueNet(torch.nn.Module):
    def __init__(self):
        super(ChessValueNet, self).__init__()
        self.C1 = torch.nn.Conv2d(6, 6, 5, padding=0)
        self.C2 = torch.nn.Conv2d(6, 16, 3, padding=1)
        self.C3 = torch.nn.Conv2d(16, 128, 4, padding=0)
        self.F1 = torch.nn.Linear(128, 64)
        self.OUT = torch.nn.Linear(64, 1)
        self.LR = torch.nn.LeakyReLU(negative_slope=0.1)

    def forward(self, x):
        x = self.C1(x)
        x = self.LR(x)
        x = self.C2(x)
        x = self.LR(x)
        x = self.C3(x)
        x = self.LR(x)
        x = x.view(x.shape[0], -1)  # flatten the feature maps into a long vector
        x = self.F1(x)
        x = self.LR(x)
        x = self.OUT(x)
        return x


In [5]:
def get_cost_function():
    return torch.nn.MSELoss()
def get_optimizer(net, lr, wd, momentum):
    optimizer =  optim.SGD(net.parameters(),lr=lr,weight_decay = wd, momentum=momentum)
    return optimizer


In [6]:
def test(net, data_loader, cost_function, model, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            # Load data into GPU
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Forward pass
            outputs = net(inputs, False)
      
            loss = cost_function(outputs, targets)

            samples+=inputs.shape[0]
            cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
    return cumulative_loss/samples


def train(net,data_loader,optimizer,cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
  
  
    for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = net(inputs, True)
    
        loss = cost_function(outputs,targets)

        loss.backward()
        optimizer.step()
    
        optimizer.zero_grad()
        samples+=inputs.shape[0]
        cumulative_loss += loss.item()
    

    return cumulative_loss/samples

In [30]:
def load_data(data_path):
    # Load the CSV file and return the data as a list of tuples
    data = []
    size = 2000
    with open(data_path, 'r') as file:
        lines = file.readlines()[1:]  # Skip header line
        for line in lines:
            size -=1
            fen, evaluation = line.strip().split(',')
            if evaluation[0] == "#":
                sign = 1
                if evaluation[1] == "-":
                    sign = -1
                evaluation = sign*10000/(1+np.log(float(evaluation[2:])))
            data.append((fen_to_tensor(fen)[0], float(evaluation)/100))
            if size <0:
                break
    return data

def get_data(batch_size, data_path, test_batch_size=100):
    data = load_data(data_path)
    # Split the dataset into training and validation data
    num_samples = len(data)
    temp_samples = int(num_samples * 0.9 + 1)
    test_samples = num_samples - temp_samples
    temp_data, test_data = torch.utils.data.random_split(data, [temp_samples, test_samples])
    
    validation_samples =  int(temp_samples * 0.4 + 1)
    training_samples = temp_samples - validation_samples
    training_data, validation_data = torch.utils.data.random_split(temp_data, [training_samples, validation_samples])
    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(validation_data, batch_size=test_batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(validation_data, batch_size=test_batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

#deep eval
def board_to_tensor(board):
    tensor = np.zeros((1,15, 8, 8), dtype=np.float32)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    prediction = game_stage(total_value(board))
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        layer = int(piece_type-1 + 3*(1+color))
        if piece_type == chess.PAWN:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value = pawn_value(n, prediction, (board, square, piece.color))
        if piece_type == chess.KING:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value += (chess.square_file(square), n, prediction)
        tensor[0,layer, chess.square_rank(square), chess.square_file(square)] = piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            tensor[0,layer, chess.square_rank(s), chess.square_file(s)] += 0.05*color
    turn = (1 if board.turn else -1)
    tensor[0, 12, 0, 0] = (0.25 if board.has_kingside_castling_rights(True) else 0)
    tensor[0, 12, -1, 0] = (-0.25 if board.has_kingside_castling_rights(False) else 0)
    tensor[0, 12, 0, -1] = (0.25 if board.has_queenside_castling_rights(True) else 0)
    tensor[0, 12, -1, -1] = (-0.25 if board.has_queenside_castling_rights(False) else 0)
    tensor[0, 13, :, :] = turn/640
    if board.is_checkmate():
        tensor[0, 14, :, :] = -turn
    return torch.from_numpy(tensor)

def fen_to_tensor(fen):
    board = chess.Board(fen)
    return(board_to_tensor(board))

In [8]:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
'''

def main(batch_size=100, 
         device='cuda:0', 
         learning_rate=0.01, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=30,
         K = 0,
         datapath = "data/random_evals.csv",
         ):
    
    train_loader, val_loader, test_loader = get_data(batch_size, datapath)
    net = ChessValueNet().to(device)
   
    optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
    cost_function = get_cost_function()

    print('Before training:')
    train_loss = test(net, train_loader, cost_function, model)
    val_loss = test(net, val_loader, cost_function, model)
    test_loss = test(net, test_loader, cost_function, model)

    print('\t Training loss {:.5f}'.format(train_loss))
    print('\t Validation loss {:.5f}'.format(val_loss))
    print('\t Test loss {:.5f}'.format(test_loss))
    print('-----------------------------------------------------')
    epoch = [0]
    training_loss = [train_loss]
    validation_loss = [val_loss]
    no_progress = 0
    last_loss = val_loss
    for e in range(epochs):
        train_loss = train(net, train_loader, optimizer, cost_function, model)
        val_loss = test(net, val_loader, cost_function, model)
        print('Epoch: {:d}'.format(e+1))
        print('\t Training loss {:.5f}'.format(train_loss))
        print('\t Validation loss {:.5f}'.format(val_loss))
        epoch.append(e+1)
        training_loss.append(train_loss)
        validation_loss.append(val_loss)
        if K != 0 :
            if (val_loss-last_loss)>-0.00002:
                no_progress +=1
            else:
                no_progress = 0
            last_loss = val_loss
            if no_progress >= K:
                print("validation loss did not decrease more than 0.00002 per epoch for "+ str(K) + " consecutive epochs, training stopped at epoch " + str(e+1))
                break

    print('-----------------------------------------------------')

    print('After training:')

    train_loss = test(net, train_loader, cost_function, model)
    val_loss = test(net, val_loader, cost_function, model)
    test_loss = test(net, test_loader, cost_function, model)

    print('\t Training loss {:.5f}'.format(train_loss))
    print('\t Validation loss {:.5f}'.format(val_loss))
    print('\t Test loss {:.5f}'.format(test_loss))
    multiPlot(epoch,[[training_loss,"training_loss"],[validation_loss,"validation_loss"]],"Epochs","loss","loss over epochs")
 
    print('-----------------------------------------------------')
    return(net)


In [14]:
def save(net):
    password = "pretrain"
    if password !="pretrain":
        print("wrong password")
        return(0)
    torch.save(net.state_dict(), "C:/Users/xiaji/école d'ing 2A/Chess AI/pretrained_net")
    print('model saved')

In [33]:

net = main(K = 3)

C:\Users\xiaji\AppData\Local\Temp/ipykernel_81384/762210738.py:14: RuntimeWarning: divide by zero encountered in log
  evaluation = sign*10000/(1+np.log(float(evaluation[2:])))


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
save(net)